In [139]:
d={"k1":"val1","k2":"val2"}

In [140]:
d

{'k1': 'val1', 'k2': 'val2'}

In [141]:
d['k1']

'val1'

In [142]:
d.k1

AttributeError: 'dict' object has no attribute 'k1'

In [ ]:
from box import ConfigBox

In [ ]:
d=ConfigBox({"k1":"val1","k2":"val2"})

In [ ]:
d.k1

'val1'

In [ ]:
from ensure import ensure_annotations

In [ ]:
@ensure_annotations
def get_predict(x:int,y:int):
    return x*y

In [ ]:
get_predict(x=3,y='a')

EnsureError: Argument y of type <class 'str'> to <function get_predict at 0x0000019EC50DE9D0> does not match annotation type <class 'int'>

In [ ]:
url ="https://drive.google.com/file/d/1ui3DZR2KVW9sO42gscrxFesZX0B4kfJK/view?usp=sharing"

In [ ]:
file_id=url.split('/')[-2]

In [ ]:
url_zip = f'https://drive.google.com/uc?id={file_id}'

In [ ]:
!pip install gdown

In [ ]:
import gdown

In [ ]:
gdown.download(url_zip,'ct_scan_kidney_images')

Downloading...
From (original): https://drive.google.com/uc?id=1ui3DZR2KVW9sO42gscrxFesZX0B4kfJK
From (redirected): https://drive.google.com/uc?id=1ui3DZR2KVW9sO42gscrxFesZX0B4kfJK&confirm=t&uuid=6fc62d7f-cb15-4bc3-bcf4-c92333c18111
To: c:\Users\hp\kidney_disease_project\research\ct_scan_kidney_images
100%|██████████| 941M/941M [19:10<00:00, 818kB/s]  


'ct_scan_kidney_images'

In [227]:
import os
%pwd
os.chdir(r'c:\Users\hp\kidney_disease_project')
%pwd

'c:\\Users\\hp\\kidney_disease_project'

In [228]:
from dataclasses import dataclass
from pathlib import Path

In [229]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    


In [230]:
from cnnClassifier.constants.config_constants import CONFIG_PATH_FILE, PARAMS_PATH_FILE
from cnnClassifier.utils.common import read_yaml,create_directories

In [231]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath =CONFIG_PATH_FILE,
        params_filepath = PARAMS_PATH_FILE):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config 


In [232]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [233]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [234]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-06 09:14:00,160: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-06 09:14:00,165: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-06 09:14:00,168: INFO: common: created directory at: artifacts]
[2025-03-06 09:14:00,170: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-06 09:14:00,172: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=d79b4573-d3ea-4813-8988-001b59121fda
To: c:\Users\hp\kidney_disease_project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [01:01<00:00, 932kB/s] 

[2025-03-06 09:15:07,139: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
